# 09: Ensamblaje y Creación de Features

**Propósito:** Este es el *notebook* final del pipeline de datos. Su objetivo es tomar los tres *datasets* maestros limpios de `data/02_processed/` y unirlos en un único *dataset* analítico listo para el modelamiento (`analytical_dataset.parquet`).

**Proceso (La "Gran Fusión"):**
1.  **Cargar:** Cargar `diputados...`, `votaciones...`, `boletines...` y el archivo externo `colegios_chile.csv`.
2.  **Enriquecer Boletines:** Aplicar **Multi-Hot Encoding** a `boletines_master` para transformar `ambitos_json` en columnas binarias (ej. `ambito_salud`, `ambito_economia`).
3.  **Enriquecer Diputados:**
    * Hacer `merge` con `colegios_chile.csv` para obtener la `dependencia` (Público/Privado).
    * Calcular `antiguedad_partido_anios` usando las fechas de inicio de período y militancia.
4.  **Ensamblar:** Unir las tres tablas enriquecidas. La tabla `votaciones_master` es nuestra "tabla de hechos" (la base) que une todo.
5.  **Guardar:** Guardar el *dataset* final en `data/03_final/`.

**Dependencias:**
* `data/02_processed/diputados_periodo_master_clean.parquet`
* `data/02_processed/votaciones_master_clean.parquet`
* `data/02_processed/boletines_master_clean.parquet`
* `data/01_raw/colegios_chile.csv` (O la ruta a tu archivo de colegios)

**Salidas (Artifacts):**
* `data/03_final/analytical_dataset.parquet`

In [1]:
import pandas as pd
import numpy as np
import logging
from pathlib import Path
import sys
import json # Para parsear los ámbitos
from sklearn.preprocessing import MultiLabelBinarizer # <-- La herramienta clave

# --- Configurar Logging ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# --- Importar lógica personalizada de /src ---
sys.path.append('../') 
try:
    # Necesitamos la función de normalizar texto para el merge de colegios
    from src.common_utils import normalize_string
except ImportError as e:
    logging.error(f"ERROR: No se pudo importar desde /src. {e}")
    raise

2025-11-05 10:32:05,973 - ERROR - ERROR: No se pudo importar desde /src. cannot import name 'Sentinel' from 'typing_extensions' (/home/angel/.local/lib/python3.10/site-packages/typing_extensions.py)


ImportError: cannot import name 'Sentinel' from 'typing_extensions' (/home/angel/.local/lib/python3.10/site-packages/typing_extensions.py)

In [ ]:
# --- 1. Configuración de Rutas y Constantes ---
ROOT = Path.cwd().parent
DATA_DIR_PROCESSED = ROOT / "data" / "02_processed"
DATA_DIR_FINAL = ROOT / "data" / "03_final"

# (Directorio de donde cargas tu archivo de colegios)
DATA_DIR_RAW = ROOT / "data" / "01_raw" 

# Asegurarse que el directorio de salida exista
DATA_DIR_FINAL.mkdir(parents=True, exist_ok=True)

# --- Archivos de Entrada ---
DIPUTADOS_FILE = DATA_DIR_PROCESSED / "diputados_master_clean.parquet"
VOTACIONES_FILE = DATA_DIR_PROCESSED / "votaciones_master_clean.parquet"
BOLETINES_FILE = DATA_DIR_PROCESSED / "boletines_master_clean.parquet"
COLEGIOS_FILE = DATA_DIR_RAW / "colegios_chile.csv" 

# --- Archivo de Salida ---
OUTPUT_FILE = DATA_DIR_FINAL / "analytical_dataset.parquet"

logging.info(f"Directorio Procesado: {DATA_DIR_PROCESSED}")
logging.info(f"Directorio Final: {DATA_DIR_FINAL}")
logging.info(f"Archivo de Salida: {OUTPUT_FILE}")

## 1. Cargar Datasets Maestros

Cargamos las tres tablas maestras de la capa `02_processed` y cualquier BBDD externa (como la de colegios).

In [ ]:
logging.info("Cargando datasets maestros...")

try:
    df_diputados = pd.read_parquet(DIPUTADOS_FILE)
    df_votaciones = pd.read_parquet(VOTACIONES_FILE)
    df_boletines = pd.read_parquet(BOLETINES_FILE)
    
    logging.info(f"Diputados cargados: {df_diputados.shape}")
    logging.info(f"Votaciones cargadas: {df_votaciones.shape}")
    logging.info(f"Boletines cargados: {df_boletines.shape}")

except FileNotFoundError as e:
    logging.error(f"ERROR: No se encontró un archivo maestro en {DATA_DIR_PROCESSED}. {e}")
    logging.error("Asegúrese de haber ejecutado los notebooks 06, 07 y 08.")
    raise

# --- Cargar BBDD Externa de Colegios ---
try:
    df_colegios_db = pd.read_csv(
        COLEGIOS_FILE,
        sep=";",              # el Mineduc casi siempre usa punto y coma
        encoding="latin-1",   # evita problemas con tildes y ñ
        on_bad_lines="skip",  # salta filas con errores
        engine="python"       # más tolerante que el parser por defecto
    )
    logging.info(f"BBDD Externa de Colegios cargada: {df_colegios_db.shape}")
except FileNotFoundError as e:
    logging.warning(f"WARNING: No se encontró el archivo de colegios en {COLEGIOS_FILE}.")
    logging.warning("La feature 'dependencia_colegio' será 'Desconocida'.")
    df_colegios_db = None

## 2. Feature Engineering: Boletines (Ámbitos Multi-Hot)

Transformamos la columna `ambitos_json` (que contiene listas) en 13 columnas binarias (Multi-Hot Encoding), una por cada ámbito temático.

In [ ]:
logging.info("Iniciando Multi-Hot Encoding de ámbitos...")

# 1. Convertir el string JSON de nuevo a una lista de Python
# (Usamos .apply(json.loads) sobre la columna que guardamos)
df_boletines['ambitos_list'] = df_boletines['ambitos_json'].apply(json.loads)

# 2. Inicializar el "codificador"
mlb = MultiLabelBinarizer()

# 3. Aplicar el Multi-Hot Encoding
df_ambitos_hot = pd.DataFrame(
    mlb.fit_transform(df_boletines['ambitos_list']),
    columns=mlb.classes_, # Nombra las columnas automáticamente
    index=df_boletines.index
)

# 4. Añadir un prefijo para claridad
df_ambitos_hot = df_ambitos_hot.add_prefix('ambito_')

# 5. Unir las nuevas columnas al DataFrame de boletines
df_boletines_enriquecido = df_boletines.join(df_ambitos_hot)

# 6. Limpiar las columnas originales y la de 'no cumple'
cols_to_drop = ['ambitos_json', 'ambitos_list']
if 'ambito_no cumple' in df_boletines_enriquecido.columns:
    cols_to_drop.append('ambito_no cumple')

df_boletines_enriquecido = df_boletines_enriquecido.drop(columns=cols_to_drop, errors='ignore')

logging.info("Features de ámbitos creadas (Multi-Hot).")
display(df_boletines_enriquecido.filter(like='ambito_').head())

## 3. Feature Engineering: Diputados (Antigüedad y Colegios)

Enriquecemos el dataset maestro de diputados con las *features* externas e internas que definimos.

In [ ]:
logging.info("Iniciando Feature Engineering en Diputados...")
df_diputados_enriquecido = df_diputados.copy()

# --- 3a. Feature: Dependencia del Colegio ---
if df_colegios_db is not None:
    logging.info("Calculando 'dependencia_colegio'...")
    # Normalizar la llave en la BBDD de colegios
    df_colegios_db['colegio_merge_key'] = df_colegios_db['NOM_RBD'].apply(normalize_string)
    
    # Seleccionar solo las columnas necesarias y eliminar duplicados
    df_colegios_lookup = df_colegios_db[['colegio_merge_key', 'COD_DEPE']].drop_duplicates()
    df_unique = (
        df_diputados_enriquecido[["colegio_merge_key"]]
        .drop_duplicates()
        .dropna()
        .reset_index(drop=True)
    )

    df_unique[["match_fuzzy", "score", "dependencia_oficial"]] = df_unique["colegio_merge_key"].apply(
    lambda x: pd.Series(find_dependencia(x, df_colegios_lookup))
    )
    
    # Hacemos el merge
    df_diputados_enriquecido = pd.merge(
        df_diputados_enriquecido,
        df_unique,
        on='colegio_merge_key',
        how='left'
    )
    df_diputados_enriquecido['dependencia_colegio'] = df_diputados_enriquecido['dependencia_oficial'].fillna('Desconocida')
    df_diputados_enriquecido = df_diputados_enriquecido.drop(columns=['dependencia_oficial'])
else:
    df_diputados_enriquecido['dependencia_colegio'] = 'Desconocida'


# --- 3b. Feature: Antigüedad en el Partido ---
logging.info("Calculando 'antiguedad_partido_anios'...")
if 'militancia_fecha_inicio' in df_diputados_enriquecido.columns:
    # Asegurar que sean datetime
    f_inicio_periodo = pd.to_datetime(df_diputados_enriquecido['periodo_fecha_inicio'], errors='coerce')
    f_inicio_militancia = pd.to_datetime(df_diputados_enriquecido['militancia_fecha_inicio'], errors='coerce')
    
    # Calcular diferencia en días y luego en años
    time_diff_days = (f_inicio_periodo - f_inicio_militancia).dt.days
    df_diputados_enriquecido['antiguedad_partido_anios'] = time_diff_days / 365.25
    
    # Manejar valores negativos (si militancia fue *después* de iniciar período)
    df_diputados_enriquecido.loc[df_diputados_enriquecido['antiguedad_partido_anios'] < 0, 'antiguedad_partido_anios'] = 0
else:
    logging.warning("No se encontró 'militancia_fecha_inicio', feature 'antiguedad' será NaN.")
    df_diputados_enriquecido['antiguedad_partido_anios'] = np.nan

logging.info("DataFrame de Diputados enriquecido.")
display(df_diputados_enriquecido[['diputado_id', 'dependencia_colegio', 'antiguedad_partido_anios']].sample(5))

In [ ]:
df_diputados_enriquecido[['diputado_id', 'colegio_raw', 'dependencia_colegio']]

In [ ]:
df_diputados_enriquecido['dependencia_colegio'].value_counts()